# Deep Learning with PyTorch: A 60 min blitz

## What is PyTorch?
It’s a Python based scientific computing package targeted at two sets of audiences:

- A replacement for numpy to use the power of GPUs
- a deep learning research platform that provides maximum flexibility and speed

## Getting Started
### Tensors
Tensors are similar to numpy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
x = torch.Tensor(5, 3); x


-2.7311e-15  4.5632e-41 -2.7311e-15
 4.5632e-41         nan  4.5632e-41
 4.4721e+21  1.6647e-41  6.7262e-44
 0.0000e+00  6.7262e-44  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00
[torch.FloatTensor of size 5x3]

In [3]:
x = torch.rand(5, 3); x


 0.1011  0.8463  0.8391
 0.3112  0.2942  0.0851
 0.7792  0.4669  0.6350
 0.1501  0.3273  0.9979
 0.9921  0.8408  0.9064
[torch.FloatTensor of size 5x3]

In [4]:
x.size()

torch.Size([5, 3])

In [5]:
y = torch.rand(5, 3)

In [6]:
x + y


 0.7551  1.6986  1.7838
 1.1031  1.0207  1.0334
 1.7389  0.5596  1.3010
 0.3530  0.8672  1.1061
 1.6770  1.2163  1.0106
[torch.FloatTensor of size 5x3]

In [7]:
torch.add(x, y)


 0.7551  1.6986  1.7838
 1.1031  1.0207  1.0334
 1.7389  0.5596  1.3010
 0.3530  0.8672  1.1061
 1.6770  1.2163  1.0106
[torch.FloatTensor of size 5x3]

In [8]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result); result


 0.7551  1.6986  1.7838
 1.1031  1.0207  1.0334
 1.7389  0.5596  1.3010
 0.3530  0.8672  1.1061
 1.6770  1.2163  1.0106
[torch.FloatTensor of size 5x3]

In [9]:
result2 = torch.add(x,y)

In [10]:
y.add_(x); y


 0.7551  1.6986  1.7838
 1.1031  1.0207  1.0334
 1.7389  0.5596  1.3010
 0.3530  0.8672  1.1061
 1.6770  1.2163  1.0106
[torch.FloatTensor of size 5x3]

Any operation that mutates a tensor in-place is post-fixed with an `_` For example: `x.copy_(y), x.t_()`, will change x.

In [11]:
# Numpy like indexing
x[:, 1]


 0.8463
 0.2942
 0.4669
 0.3273
 0.8408
[torch.FloatTensor of size 5]

## Numpy Bridge
Converting a torch Tensor to a numpy array and vice versa is a breeze.

The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other.

In [12]:
a = torch.ones(5); a


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

In [13]:
b = a.numpy(); b

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [14]:
a.add_(1); a


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

In [15]:
b

array([ 2.,  2.,  2.,  2.,  2.], dtype=float32)

In [16]:
import numpy as np

In [17]:
a = np.ones(5); a

array([ 1.,  1.,  1.,  1.,  1.])

In [18]:
b = torch.from_numpy(a); b


 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]

In [19]:
np.add(a, 1, out=a)

array([ 2.,  2.,  2.,  2.,  2.])

In [20]:
b


 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]

All the Tensors on the CPU except a CharTensor support converting to NumPy and back.

## CUDA Tensors
Tensors can be moved onto GPU using the .cuda function.

In [21]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x+y)


 0.8562  2.5449  2.6229
 1.4143  1.3149  1.1185
 2.5180  1.0265  1.9359
 0.5031  1.1945  2.1040
 2.6691  2.0571  1.9169
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]



## Autograd: automatic differentiation

Central to all neural networks in PyTorch is the autograd package. Let’s first briefly visit this, and we will then go to training our first neural network.

The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

### Variable
`autograd.Variable` is the central class of the package. It wraps a Tensor, and supports nearly all of operations defined on it. Once you finish your computation you can call `.backward()` and have all the gradients computed automatically.

You can access the raw tensor through the `.data` attribute, while the gradient w.r.t. this variable is accumulated into `.grad`.

There’s one more class which is very important for autograd implementation - a `Function`.

`Variable` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each variable has a `.grad_fn` attribute that references a `Function` that has created the `Variable` (except for Variables created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Variable. If `Variable` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `grad_output` argument that is a tensor of matching shape.

In [22]:
x = Variable(torch.ones(2,2), requires_grad=True); x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [23]:
y = x + 2; y

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [24]:
x.data


 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [25]:
print(x.grad_fn)

None


`y` was created as a result of an operation, so it has a `grad_fn`.

In [26]:
print(y.grad_fn)

In [27]:
z = y * y * 3
out = z.mean()
print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



In [28]:
x = torch.randn(3)
x = Variable(x, requires_grad = True)
y = x * 2

In [29]:
while y.data.norm() < 1000:
    y = y * 2

print(y)

Variable containing:
-1247.1676
  890.2509
-1168.2587
[torch.FloatTensor of size 3]



In [30]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  409.6000
 4096.0000
    0.4096
[torch.FloatTensor of size 3]



## Neural Networks

Neural networks can be constructed using the `torch.nn` package.

Now that you had a glimpse of `autograd`, `nn` depends on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and a method `forward(input)` that returns the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: 

weight = weight - learning_rate * gradient

### Define the network

In [31]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [32]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [33]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
-0.0344  0.0544  0.0985  0.1253  0.0158  0.0222  0.1166 -0.0159 -0.0856  0.1123
[torch.FloatTensor of size 1x10]



In [34]:
net.zero_grad()
out.backward(torch.randn(1, 10), retain_graph=True)

Recap:

- torch.Tensor - A multi-dimensional array.
- autograd.Variable - Wraps a Tensor and records the history of operations applied to it. Has the same API as a Tensor, with some additions like backward(). Also holds the gradient w.r.t. the tensor.
- nn.Module - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
- nn.Parameter - A kind of Variable, that is automatically registered as a parameter when assigned as an attribute to a Module.
- autograd.Function - Implements forward and backward definitions of an autograd operation. Every Variable operation, creates at least a single Function node, that connects to functions that created a Variable and encodes its history.

In [35]:
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

Variable containing:
 38.0814
[torch.FloatTensor of size 1]



In [36]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [37]:
net.zero_grad()

In [38]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -1.0619
 -2.8369
 -0.4189
 -5.6938
 -0.2513
  0.7978
[torch.FloatTensor of size 6]



In [39]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [40]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

# PyTorch Fundamentals - Matrices

In [41]:
import numpy as np

In [42]:
arr = [[1, 2], [3, 4]]; arr

[[1, 2], [3, 4]]

In [43]:
np.array(arr)

array([[1, 2],
       [3, 4]])

In [44]:
import torch

In [45]:
# Convert to PyTorch Tensor
torch.Tensor(arr)


 1  2
 3  4
[torch.FloatTensor of size 2x2]

In [46]:
np.ones((2,2))

array([[ 1.,  1.],
       [ 1.,  1.]])

In [47]:
torch.ones((2,2))


 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [48]:
np.random.rand(2,2)

array([[ 0.51610964,  0.18910499],
       [ 0.1142441 ,  0.93675801]])

In [49]:
torch.rand(2,2)


 0.5421  0.4469
 0.5524  0.0700
[torch.FloatTensor of size 2x2]

## Seed for reproducibility

In [50]:
# Seed
np.random.seed(0)
np.random.rand(2,2)

array([[ 0.5488135 ,  0.71518937],
       [ 0.60276338,  0.54488318]])

In [51]:
# No seed
np.random.rand(2,2)

array([[ 0.4236548 ,  0.64589411],
       [ 0.43758721,  0.891773  ]])

In [52]:
# Torch Seed
torch.manual_seed(0)
torch.rand(2,2)


 0.5488  0.5928
 0.7152  0.8443
[torch.FloatTensor of size 2x2]

In [53]:
# Torch Seed
torch.manual_seed(0)
torch.rand(2,2)


 0.5488  0.5928
 0.7152  0.8443
[torch.FloatTensor of size 2x2]

In [54]:
# No Seed
torch.rand(2,2)


 0.6028  0.8579
 0.5449  0.8473
[torch.FloatTensor of size 2x2]

In [55]:
# No Seed
torch.rand(2,2)


 0.4237  0.6236
 0.6459  0.3844
[torch.FloatTensor of size 2x2]

### Seed for GPU is different

In [56]:
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

## NumPy to Torch bridge

In [67]:
new_array = np.ones((2,2), dtype=np.float32)

In [68]:
torch.from_numpy(new_array)


 1  1
 1  1
[torch.FloatTensor of size 2x2]

## Torch to NumPy bridge

In [69]:
tt = torch.ones((2,2))

In [70]:
tt.numpy()

array([[ 1.,  1.],
       [ 1.,  1.]], dtype=float32)

In [71]:
type(tt)

torch.FloatTensor

In [72]:
type(tt.numpy())

numpy.ndarray

In [73]:
tensor_cpu = torch.ones((2,2))

In [74]:
if torch.cuda.is_available():
    tensor_cpu.cuda()

In [75]:
tensor_cpu.cpu()


 1  1
 1  1
[torch.FloatTensor of size 2x2]

# Tensor Operations

## Resizing Tensor

In [79]:
a = torch.ones(2,2); a


 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [80]:
a.size()

torch.Size([2, 2])

In [81]:
a.view(4)


 1
 1
 1
 1
[torch.FloatTensor of size 4]

In [82]:
a.view(4).size()

torch.Size([4])

## Element-wise addition